In [1]:
import pandas as pd
import numpy as np
import random
float32ify = lambda df: df.astype({c: np.float32 for c in df.select_dtypes(include='float64').columns})

In [2]:
marker_path = "markers.csv"
true_prop_path = "gbm_neftel_true_prop.csv"
gexp_path = "gbm_neftel_gexp.csv"
test_split = 0.3

In [3]:
markers = pd.read_csv(marker_path, index_col=0, header=None)
markers.index.rename("cell_type", inplace=True)
markers

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
cell_type,,,,,,,,,,,,,,,,,,,,,
metasig1,SOX4,DCX,CD24,ELAVL4,STMN2,SOX11,TUBB3,MIAT,STMN1,HMP19,...,GPC2,FBLIM1,DBN1,HN1,LYZ,KIF5C,KIF21B,DLX1,MEX3A,ORC4
metasig2,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A4GALT,A4GNT,AA06,...,ABCA13,ABCA17P,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,ABCA7,ABCA8,ABCA9
metasig3,ABCB1,ABCB10,ABCB11,ABCB4,ABCB5,ABCB6,ABCB7,ABCB8,ABCB9,ABCC1,...,ABHD12,ABHD12B,ABHD13,ABHD14A,ABHD14A-ACY1,ABHD14B,ABHD15,ABHD16A,ABHD16B,ABHD2
metasig4,ABHD3,ABHD4,ABHD5,ABHD6,ABHD8,ABI1,ABI2,ABI3,ABI3BP,ABL1,...,ACBD3,ACBD4,ACBD5,ACBD6,ACBD7,ACCS,ACCSL,ACD,ACE,ACE2
metasig5,ACER1,ACER2,ACER3,ACHE,ACIN1,ACLY,ACMSD,ACO1,ACO2,ACOT1,...,ACSL5,ACSL6,ACSM1,ACSM2A,ACSM2B,ACSM3,ACSM4,ACSM5,ACSS1,ACSS2
Macrophage,ACTB,ACTG1,ACTL6A,ACTN1,ACTN4,ACTR10,ACTR1A,ACTR1B,ACTR2,ACTR3,...,ADRM1,ADSL,ADSS,AEN,AFF1,AFF4,AFG3L1P,AFG3L2,AFMID,AFTPH
Oligodendrocyte,ACTN2,ACYP1,ADA,ADAMTS1,ADAMTS18,ADAMTS4,ADAP1,ADARB2,ADCK1,ADCK4,...,AK1,AK2,AK3,AK5,AKAP10,AKAP11,AKAP6,AKAP8,AKAP8L,AKAP9
T-cell,ACTA2,ACTR5,ADAM19,ADAT1,ADHFE1,ADORA2A,AEBP2,AES,AGAP2,AGPAT2,...,ALKBH7,ALOX5,ALOX5AP,AMD1,AMICA1,AMMECR1L,AMPD2,AMT,AMZ2,AMZ2P1


In [4]:
markers_all = pd.Index(markers.to_numpy().flatten())
markers_all

Index(['SOX4', 'DCX', 'CD24', 'ELAVL4', 'STMN2', 'SOX11', 'TUBB3', 'MIAT',
       'STMN1', 'HMP19',
       ...
       'ALKBH7', 'ALOX5', 'ALOX5AP', 'AMD1', 'AMICA1', 'AMMECR1L', 'AMPD2',
       'AMT', 'AMZ2', 'AMZ2P1'],
      dtype='object', length=400)

In [79]:
gexp = float32ify(pd.read_csv(gexp_path, index_col=["patient", "pseudopatient"]))
gexp

SOX4        DCX       CD24    ELAVL4     STMN2  \
patient pseudopatient                                                        
MGH100  0               8.906667  10.710972  12.467972  8.324722  1.454139   
        1               9.629778   4.658750  12.532972  5.764972  2.094528   
        2               7.661194   6.442056  32.603001  8.949667  0.806611   
        3              10.333400   6.470685  21.368458  9.211600  2.804914   
MGH101  0              14.105525   0.027983   0.388746  2.357322  0.003220   
...                          ...        ...        ...       ...       ...   
BT920   0              19.959106   4.506842  27.127209  2.032912  1.875341   
        1              29.958492  12.920451  37.814121  2.431114  1.988166   
        2              15.430511   2.756576  16.734137  0.776071  0.049795   
        3              13.431654   4.253436  34.586487  0.552494  5.001658   
        4              20.165047   5.552359  34.581341  0.132859  2.537606   

                           SOX11       TUBB3      MIAT       STMN1      HMP19  \
patient pseudopatient                                                           
MGH100  0               4.525167   18.628973  0.222278   56.745834   2.876417   
        1               4.478028   11.419361  0.388361   75.220108   1.702139   
        2               3.801250   14.506805  0.480222   47.619221   1.867528   
        3               5.049114   21.148972  1.014971   87.135025   4.091686   
MGH101  0               7.525915    8.979526  0.000000   12.770967   0.000000   
...                          ...         ...       ...         ...        ...   
BT920   0              14.440137   89.119408  6.541327  253.173187   4.772989   
        1              25.973110  181.057724  8.957916  302.318573  22.015562   
        2              16.571514   95.912048  3.941407  214.161728   5.831143   
        3              20.166307  121.156601  3.910415  233.842072   3.601452   
        4              17.211956  124.374939  5.195744  202.506119   9.701889   

                       ...    ALKBH7     ALOX5    ALOX5AP       AMD1  AMICA1  \
patient pseudopatient  ...                                                     
MGH100  0              ...  5.795944  0.000000   0.000000  11.756722     0.0   
        1              ...  4.038750  0.000000   0.247639  15.766055     0.0   
        2              ...  3.922500  0.000000   0.000000  15.758361     0.0   
        3              ...  4.669657  0.052686   0.000000  14.742314     0.0   
MGH101  0              ...  6.810898  3.482220  47.965660  14.207966     0.0   
...                    ...       ...       ...        ...        ...     ...   
BT920   0              ...  7.581811  0.511964   0.352375  15.791529     0.0   
        1              ...  6.013845  1.050514  27.290220  12.181211     0.0   
        2              ...  6.655757  3.679496  56.544563  11.722325     0.0   
        3              ...  6.668577  1.954238  50.547306  16.556133     0.0   
        4              ...  9.828856  1.149196  40.591133  13.194280     0.0   

                       AMMECR1L     AMPD2       AMT       AMZ2    AMZ2P1  
patient pseudopatient                                                     
MGH100  0              2.112917  2.768833  0.994806  12.766556  3.498139  
        1              2.527528  6.025333  0.324500  12.117056  4.053333  
        2              1.367417  4.787667  0.766778  10.850889  3.613055  
        3              3.447572  3.692743  0.365971   8.110343  4.285914  
MGH101  0              0.609119  8.385746  0.001356  10.487712  3.093237  
...                         ...       ...       ...        ...       ...  
BT920   0              2.421005  5.292770  0.158664   9.288868  2.471902  
        1              1.789012  2.492109  1.989592   5.219711  0.990267  
        2              1.569103  7.538952  1.967734   3.608749  2.119743  
        3              1.434548  1.616997  0.931141   2.557757  1.912484  
        4         

In [80]:
true_prop = pd.read_csv(true_prop_path, index_col=["patient", "pseudopatient"])
true_prop

metasig1  metasig2  metasig3  metasig4  metasig5  \
patient pseudopatient                                                     
MGH100  0              0.083333  0.277778  0.111111  0.333333  0.194444   
        1              0.055556  0.416667  0.194444  0.138889  0.194444   
        2              0.083333  0.416667  0.083333  0.305556  0.111111   
        3              0.085714  0.114286  0.171429  0.342857  0.285714   
MGH101  0              0.000000  0.258621  0.068966  0.327586  0.068966   
...                         ...       ...       ...       ...       ...   
BT920   0              0.181818  0.030303  0.000000  0.606061  0.151515   
        1              0.371429  0.000000  0.028571  0.371429  0.142857   
        2              0.175000  0.000000  0.150000  0.475000  0.000000   
        3              0.236842  0.052632  0.026316  0.500000  0.026316   
        4              0.194444  0.027778  0.083333  0.472222  0.083333   

                       Macrophage  Oligodendrocyte  T-cell  
patient pseudopatient                                       
MGH100  0                0.000000              0.0     0.0  
        1                0.000000              0.0     0.0  
        2                0.000000              0.0     0.0  
        3                0.000000              0.0     0.0  
MGH101  0                0.275862              0.0     0.0  
...                           ...              ...     ...  
BT920   0                0.030303              0.0     0.0  
        1                0.085714              0.0     0.0  
        2                0.200000              0.0     0.0  
        3                0.157895              0.0     0.0  
        4                0.138889              0.0     0.0  

[100 rows x 8 columns]

In [81]:
patients = gexp.index.get_level_values(0).unique().tolist()
random.shuffle(patients)
train_patients = patients[round(test_split*len(patients)):]
test_patients = patients[:round(test_split*len(patients))]

In [82]:
train_x = gexp.loc[train_patients]
test_x = gexp.loc[test_patients]
train_y = true_prop.loc[train_patients]
test_y = true_prop.loc[test_patients]